In [15]:
%load_ext autoreload

In [20]:
%autoreload 2

In [1]:
from src.parsers import mimic, hirid
from src.modeling import discovery, plots, querier
from src.utils import constants

In [2]:
import sys
# import win32com.client
import os
import pandas as pd

def setup_io_config(root_path):
    """
    Input - Output config. Add dataset paths
    :root_path -> Repo path which contains 'data' and 'res' folders
    """

    # MIMIC
    is_shortcut = True if "data.lnk" in os.listdir(root_path) else False 
    
    if (is_shortcut):
        path_shortcut =  os.path.join(root_path, "data.lnk")
        shell = win32com.client.Dispatch("WScript.Shell")
        mimic_data = shell.CreateShortCut(path_shortcut).Targetpath
    else:
        mimic_data = os.path.join(f"{root_path}", "data") 
    mimic_path = os.path.join(f"{root_path}", "results")

    # HIRID
    hirid_data = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1/raw_stage/'
    hirid_path = f'{root_path}/data/hirid-a-high-time-resolution-icu-dataset-1.1.1'
    
    return mimic_data, mimic_path, hirid_data, hirid_path

def setup_stratification_config():
    gender="MF"
    age_b=40
    age_a=80 
    ethnicity="WHITE" 
    lab_mapping= constants.LAB_MAPPING
    b_w = [(0,6), (6,12)]
    a_w = [(0,4), (4,8), (8,12)]
    before_windows = b_w
    after_windows = a_w
    return gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows

In [3]:
# IO Config
# root_path ="C:\\Users\\danco\\My Drive\\Master\\Datasets\\MIMIC iii"
root_path = "/Users/pavan/Library/CloudStorage/GoogleDrive-f20190038@hyderabad.bits-pilani.ac.in/My Drive/TAU/Code/DrugLab"
data, res, raw_path, res_path = setup_io_config(root_path=root_path)

# Stratification Config
gender, age_a, age_b, ethnicity, lab_mapping, before_windows, after_windows = setup_stratification_config()

In [4]:
res_dict = {}
lab = list(constants.LAB_MAPPING.keys())[0]
print(f"Parsing {lab} data....")
# MIMIC
mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=True, load_raw_chartevents=False, lab_parts=(0, 15), lab=lab)
m_labs = m_labs.reset_index().drop(columns=["index"])
print(f"Done parsing {lab}.")

Parsing Heart Rate data....
Loading med data...
Generate med data....


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:51: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ROW_ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  m_p_df = pd.merge(inputevents_mv, med_data, how="inner", on="ITEMID")


Generated med data.
Loaded med data.
Load 1st and 2nd medication data...
Loaded 1st and 2nd medication data.
Load Lab data...
Generate lab data from labevents...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:138: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  labevents = pd.read_csv(os.path.join(self.data, constants.MIMIC_III_PREPROCESSED_PATH, constants.MIMIC_III_LABEVENT_PREPROCESSED))


Generated lab data from labevents.
Generate lab data from chartevents...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...


...

Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:194: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Loading Heart Rate data from CHARTEVENTS...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:194: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Loading Heart Rate data from CHARTEVENTS...
Generated lab data from chartevents.
Loaded Lab data.
Done parsing Heart Rate.


In [5]:
print(f"Starting pairs generation for {lab}....")
## Querier
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    t_labs=m_labs, 
    t_med1=m_med1.iloc[:10], 
    t_med2=m_med2,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
# query pairs for all medication and lab tests
t2 = mimic_data_querier.generate_med_lab_data(before_windows, after_windows, lab_parts=f"{lab}_(0,15)")

Starting pairs generation for Heart Rate....


KeyError: 'before_mean_(0, 6)'

In [ ]:
t2

In [ ]:
res_dict = {}
for lab in constants.LAB_MAPPING.keys():
    print(f"Parsing {lab} data....")
    # MIMIC
    mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
    m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=True, load_raw_chartevents=False, lab_parts=(0, 15), lab=lab)
    m_labs = m_labs.reset_index().drop(columns=["index"])
    print(f"Done parsing {lab}.")
    
    print(f"Starting pairs generation for {lab}....")
    ## Querier
    mimic_data_querier = querier.DatasetQuerier(
        data = data,
        res = res,
        t_labs=m_labs, 
        t_med1=m_med1, 
        t_med2=m_med2,
        gender=gender, 
        age_b=age_b, 
        age_a=age_a, 
        ethnicity=ethnicity, 
        lab_mapping=lab_mapping
    )
    # query pairs for all medication and lab tests
    res_dict[lab] = mimic_data_querier.generate_med_lab_data(before_windows, after_windows, lab_parts=f"{lab}_(0,15)")
    print(f"Pairs generated for {lab}.")
    

In [4]:
# MIMIC
mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=True, load_raw_chartevents=False, lab_parts=(0, 15))

Loading med data...
Generate med data....


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:51: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ROW_ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  m_p_df = pd.merge(inputevents_mv, med_data, how="inner", on="ITEMID")


Generated med data.
Loaded med data.
Load 1st and 2nd medication data...
Loaded 1st and 2nd medication data.
Load Lab data...
Generate lab data from labevents...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:138: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  labevents = pd.read_csv(os.path.join(self.data, constants.MIMIC_III_PREPROCESSED_PATH, constants.MIMIC_III_LABEVENT_PREPROCESSED))


Generated lab data from labevents.
Generate lab data from chartevents...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on

Generated lab data from chartevents.


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:213: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  merged_chart_lab_events = merged_chart_lab_events.groupby(["HADM_ID", "ADMITTIME", "CHARTTIME", "VALUENUM", "MIMICExtractName"]).nth(0).reset_index()


Loaded Lab data.


In [5]:
m_labs

,HADM_ID,ADMITTIME,CHARTTIME,VALUENUM,ITEMID,ADMISSION_LOCATION,ADMISSION_TYPE,AGE,DEATHTIME,DIAGNOSIS,...,LANGUAGE,MARITAL_STATUS,RELIGION,SUBJECT_ID,VALUE,VALUEUOM,ROW_ID_x,TABLE,LabTimeFromAdmit,hours_in
318,100003.0,2150-04-17 15:34:00,2150-04-17 17:00:00,36.0,Diastolic blood pressure,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,36.0,mmHg,12554654,CHARTEVENTS,0 days 01:26:00,1.433333
319,100003.0,2150-04-17 15:34:00,2150-04-17 17:00:00,83.0,Systolic blood pressure,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,83.0,mmHg,12554653,CHARTEVENTS,0 days 01:26:00,1.433333
320,100003.0,2150-04-17 15:34:00,2150-04-17 17:51:00,71.0,Heart Rate,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,71.0,bpm,12554658,CHARTEVENTS,0 days 02:17:00,2.283333
321,100003.0,2150-04-17 15:34:00,2150-04-17 17:51:00,96.0,Pulse oximetry,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,96.0,%,12554659,CHARTEVENTS,0 days 02:17:00,2.283333
322,100003.0,2150-04-17 15:34:00,2150-04-17 17:52:00,11.0,Respiratory rate,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,11.0,insp/min,12554660,CHARTEVENTS,0 days 02:18:00,2.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17390186,199948.0,2102-02-22 13:28:00,2102-02-26 13:00:00,97.0,Systolic blood pressure,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,97.0,mmHg,10119190,CHARTEVENTS,3 days 23:32:00,95.533333
17390187,199948.0,2102-02-22 13:28:00,2102-02-26 13:00:00,122.0,Systolic blood pressure,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,122.0,mmHg,10119195,CHARTEVENTS,3 days 23:32:00,95.533333
17390188,199948.0,2102-02-22 13:28:00,2102-02-26 14:00:00,52.0,Diastolic blood pressure,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,52.0,mmHg,10119199,CHARTEVENTS,4 days 00:32:00,96.533333
17390189,199948.0,2102-02-22 13:28:00,2102-02-26 14:00:00,94.0,Systolic blood pressure,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,94.0,mmHg,10119198,CHARTEVENTS,4 days 00:32:00,96.533333


In [6]:
m_labs = m_labs.reset_index().drop(columns=["index"])
m_labs

,HADM_ID,ADMITTIME,CHARTTIME,VALUENUM,ITEMID,ADMISSION_LOCATION,ADMISSION_TYPE,AGE,DEATHTIME,DIAGNOSIS,...,LANGUAGE,MARITAL_STATUS,RELIGION,SUBJECT_ID,VALUE,VALUEUOM,ROW_ID_x,TABLE,LabTimeFromAdmit,hours_in
0,100003.0,2150-04-17 15:34:00,2150-04-17 17:00:00,36.0,Diastolic blood pressure,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,36.0,mmHg,12554654,CHARTEVENTS,0 days 01:26:00,1.433333
1,100003.0,2150-04-17 15:34:00,2150-04-17 17:00:00,83.0,Systolic blood pressure,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,83.0,mmHg,12554653,CHARTEVENTS,0 days 01:26:00,1.433333
2,100003.0,2150-04-17 15:34:00,2150-04-17 17:51:00,71.0,Heart Rate,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,71.0,bpm,12554658,CHARTEVENTS,0 days 02:17:00,2.283333
3,100003.0,2150-04-17 15:34:00,2150-04-17 17:51:00,96.0,Pulse oximetry,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,96.0,%,12554659,CHARTEVENTS,0 days 02:17:00,2.283333
4,100003.0,2150-04-17 15:34:00,2150-04-17 17:52:00,11.0,Respiratory rate,EMERGENCY ROOM ADMIT,EMERGENCY,60.0,NaN,UPPER GI BLEED,...,ENGL,SINGLE,NOT SPECIFIED,54610,11.0,insp/min,12554660,CHARTEVENTS,0 days 02:18:00,2.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5480665,199948.0,2102-02-22 13:28:00,2102-02-26 13:00:00,97.0,Systolic blood pressure,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,97.0,mmHg,10119190,CHARTEVENTS,3 days 23:32:00,95.533333
5480666,199948.0,2102-02-22 13:28:00,2102-02-26 13:00:00,122.0,Systolic blood pressure,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,122.0,mmHg,10119195,CHARTEVENTS,3 days 23:32:00,95.533333
5480667,199948.0,2102-02-22 13:28:00,2102-02-26 14:00:00,52.0,Diastolic blood pressure,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,52.0,mmHg,10119199,CHARTEVENTS,4 days 00:32:00,96.533333
5480668,199948.0,2102-02-22 13:28:00,2102-02-26 14:00:00,94.0,Systolic blood pressure,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,74.0,NaN,RECTAL PROLAPSE,...,ENGL,DIVORCED,NOT SPECIFIED,49225,94.0,mmHg,10119198,CHARTEVENTS,4 days 00:32:00,96.533333


In [9]:
m_labs.ITEMID.value_counts(ascending=True)

Temperature (C)               31174
Oxygen saturation             34479
Temperature (F)              194146
Diastolic blood pressure     951084
Systolic blood pressure      953738
Pulse oximetry               991343
Heart Rate                  1025638
Respiratory rate            1299068
Name: ITEMID, dtype: int64

In [10]:
res_dict = {}
for labs in m_labs.ITEMID.value_counts(ascending=True).keys():
    print(f"Starting pairs generation for {labs}....")
    t_m_labs = m_labs[m_labs["ITEMID"]==labs]
    ## Querier
    mimic_data_querier = querier.DatasetQuerier(
        data = data,
        res = res,
        t_labs=t_m_labs, 
        t_med1=m_med1, 
        t_med2=m_med2,
        gender=gender, 
        age_b=age_b, 
        age_a=age_a, 
        ethnicity=ethnicity, 
        lab_mapping=lab_mapping
    )
    # query pairs for all medication and lab tests
    res_dict[labs] = mimic_data_querier.generate_med_lab_data(before_windows, after_windows, lab_parts=f"{labs}_(0,15)")
    print(f"Pairs generated for {labs}.")

Starting pairs generation for Temperature (C)....


KeyboardInterrupt: 

In [8]:
## Querier
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    t_labs=m_labs, 
    t_med1=m_med1, 
    t_med2=m_med2,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
# query pairs for all medication and lab tests
m_final_lab_med_data = mimic_data_querier.generate_med_lab_data(before_windows, after_windows, lab_parts=(0,15))

KeyboardInterrupt: 

In [ ]:
m_final_lab_med_data

In [10]:
m_labs.to_csv("temp_mimic_{lab_parts}_labs.csv")
m_med1.to_csv("temp_mimic_{lab_parts}_med1.csv")
m_med2.to_csv("temp_mimic_{lab_parts}_med2.csv")

In [11]:
# MIMIC
mimic_parser = mimic.MIMICParser(data=data, res=res, gender=gender, age_b=age_b, age_a=age_a, ethnicity=ethnicity, load="AUTOMATIC_MAPPING_MIMIC")
m_med1, m_med2, m_labs = mimic_parser.parse(use_pairs=False, load_from_raw=True, load_raw_chartevents=False, lab_parts=(15,30))

Loading med data...
Generate med data....


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:51: FutureWarning: Passing 'suffixes' which cause duplicate columns {'ROW_ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  m_p_df = pd.merge(inputevents_mv, med_data, how="inner", on="ITEMID")


Generated med data.
Loaded med data.
Load 1st and 2nd medication data...
Loaded 1st and 2nd medication data.
Load Lab data...
Generate lab data from labevents...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:138: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  labevents = pd.read_csv(os.path.join(self.data, constants.MIMIC_III_PREPROCESSED_PATH, constants.MIMIC_III_LABEVENT_PREPROCESSED))


Generated lab data from labevents.
Generate lab data from chartevents...


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.concat([pd.read_csv(path) for path in res_paths], ignore_index=True) ## concatenation of output from each chunk
/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:184: DtypeWarning: Columns (9) have mixed types. Specify dtype option on imp

Generated lab data from chartevents.


/Users/pavan/Desktop/TAU/DrugLab/src/parsers/mimic.py:213: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  merged_chart_lab_events = merged_chart_lab_events.groupby(["HADM_ID", "ADMITTIME", "CHARTTIME", "VALUENUM", "MIMICExtractName"]).nth(0).reset_index()


Loaded Lab data.


In [12]:
## Querier
mimic_data_querier = querier.DatasetQuerier(
    data = data,
    res = res,
    t_labs=m_labs, 
    t_med1=m_med1, 
    t_med2=m_med2,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
# query pairs for all medication and lab tests
m_final_lab_med_data = mimic_data_querier.generate_med_lab_data(before_windows, after_windows, lab_parts=(15,30))

/Users/pavan/Desktop/TAU/DrugLab/src/modeling/querier.py:119: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  temp = temp.apply(lambda r : self.get_vals(r, t_labs, t_med1, t_med2, before_windows, after_windows), axis=1)


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
## Discovery Analysis for the queried medication and lab test pairs in the chosen before and after windows
analyzer = discovery.ClinicalDiscoveryAnalysis(m_final_lab_med_data)
pvals_med_lab = analyzer.analyze(before_windows, after_windows)
sig_med_lab = analyzer.generate_significant(pvals_med_lab.dropna(subset=["TTest Paired"]))

In [ ]:

## Plots
plotter = plots.ClinicalPlotAnalysis(
    data = data,
    res = res,
    gender=gender, 
    age_b=age_b, 
    age_a=age_a, 
    ethnicity=ethnicity, 
    lab_mapping=lab_mapping
)
m_corrs_data_df = plotter.plot(m_final_lab_med_data, m_labs, before_windows=before_windows, after_windows=after_windows)